In [4]:
!pip install transformers sentence-transformers faiss-cpu PyPDF2 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 680.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

# Processing PDF Content with Text Embeddings and Summarization

In this notebook, we demonstrate:
1. Reading and processing a PDF file.
2. Splitting text into manageable chunks.
3. Generating embeddings for the chunks using SentenceTransformers.
4. Using FAISS for efficient similarity search.
5. Summarizing the most relevant chunks with Hugging Face models.

## 1. Reading a PDF File
We use the `PyPDF2` library to extract text from a PDF file. The text from all pages is concatenated into a single string.

In [19]:
from PyPDF2 import PdfReader

# Function to read a PDF file
def read_pdf(file_path):
    reader = PdfReader(file_path)
    content = ""
    for page in reader.pages:
        content += page.extract_text() + "\n"  # Append text from each page
    return content

# Replace with your PDF file path

file_path = "/content/documents/LLM.pdf"
pdf_content = read_pdf(file_path)

In [20]:
pdf_content

'A Beginner’s Guide to \nLarge Language Models\nPart 1\nContributors:\nAnnamalai Chockalingam\nAnkur Patel\nShashank Verma\nTiffany Yeung\nA Beginner’s Guide to Large Language Models  2 Table of Contents Preface ....................................................................................................................................................... 3 Glossary ...................................................................................................................................................... 5 Introduction to LLMs.................................................................................................................................. 8 What Are Large Language Models (LLMs)? .......................................................................................... 8 Foundation Language Models vs. Fine-Tuned Language Models ...................................................... 11 Evolution of Large Language Models ....................................

## 2. Splitting Text into Chunks
To handle large text efficiently, we split the content into smaller chunks of a specified size. Each chunk contains a specific number of words.

In [21]:
# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=300):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Split the text into chunks
chunks = split_text_into_chunks(pdf_content, chunk_size=64)

In [23]:


122*64

7808

In [33]:

len(chunks)

# chunks[25]

122

## 3. Generating Text Embeddings
We use the `SentenceTransformer` model to generate embeddings for each chunk of text. Embeddings are numerical representations of text that capture semantic meaning.

In [32]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Function to generate embeddings for text chunks
def embed_text_chunks(chunks, embedding_model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(embedding_model_name)
    embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
    return embeddings

# Generate embeddings using a specified model
embedding_model_name = "all-mpnet-base-v2"
embeddings = embed_text_chunks(chunks, embedding_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
embeddings.shape

(122, 768)

In [ ]:
embeddings.shape

(122, 768)

In [41]:
# embeddings[121]

## 4. Building a FAISS Index for Similarity Search
FAISS (Facebook AI Similarity Search) is used to efficiently search for the most similar text chunks based on their embeddings.

In [42]:
import faiss

# Function to build a FAISS index
def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Build the FAISS index
faiss_index = build_faiss_index(embeddings)

## 5. Search the FAISS Index
We perform a similarity search on the FAISS index to retrieve the chunks most relevant to a given query. The query is also converted into an embedding for this purpose.

In [44]:
# RETRIEVAL of information

# Define a query and generate its embedding
query = "Foundation Language Models vs. Fine-Tuned Language Models"
query_embedding = SentenceTransformer(embedding_model_name).encode([query], convert_to_numpy=True)
# print(query_embedding)
# Retrieve top-3 closest chunks
distances, indices = faiss_index.search(query_embedding, k=1)
response_chunks = '\n'.join([chunks[i] for i in indices[0]])
print(response_chunks)

fine-tuned model that specializes in performing tasks in a narrowly scoped domain. A Beginner’s Guide to Large Language Models 11 Foundation Language Models vs. Fine-Tuned Language Models Foundation language models, such as the aforementioned MT-NLG and GPT-3, are what is usually referred to when discussing LLMs. They’re trained on vast amounts of data and can perform a wide variety of NLP tasks, from answering


## 6. Summarizing the Retrieved Chunks/Responding to query with retrieved chunk
We use the `bart-large-cnn` model to summarize the most relevant chunks retrieved from the FAISS index.

In [46]:
from transformers import pipeline

# Initialize the summarization pipeline
summarize_model = pipeline("summarization", model="facebook/bart-large-cnn")

# Summarize the retrieved chunks
summary = summarize_model(response_chunks, max_length=50, min_length=30, do_sample=False)
print(summary)

Device set to use cpu


[{'summary_text': 'Foundation language models are what is usually referred to when discussing LLMs. They’re trained on vast amounts of data and can perform a wide variety of NLP tasks. Fine-Tuned Language Models are trained on a narrowly'}]


In [5]:
!pip install groq

# Integrating Groq API with LLaMA for Chat Completions

This notebook demonstrates how to interact with the **Groq API** to generate chat completions using the `llama-3.3-70b-specdec` model. The Groq platform provides access to advanced NLP models for generating detailed responses based on user inputs.

## Step 1: Importing Required Libraries
We start by importing the necessary libraries. Here, we use `userdata` from Google Colab to securely store and retrieve sensitive information like API keys.

In [6]:
from google.colab import userdata

## Step 2: Retrieving the API Key
We retrieve the `GROQ_API_KEY` from Colab's secure storage. This key is necessary to authenticate and interact with the Groq API.

In [7]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

## Step 3: Initializing the Groq Client
Using the `groq` library, we initialize the client with the retrieved API key. This allows us to interact with the models available on the platform.

In [8]:
import requests


url = "https://api.groq.com/test_endpoint"
headers = {"Authorization": f"Bearer {GROQ_API_KEY}"}
response = requests.get(url, headers=headers)
print(response.json())


{'error': {'message': 'Unknown request URL: GET /test_endpoint. Please check the URL for typos, or see the docs at https://console.groq.com/docs/', 'type': 'invalid_request_error', 'code': 'unknown_url'}}


In [9]:
from groq import Groq

# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)

## Step 4: Sending a Chat Completion Request
We send a request to the Groq API to generate a response using the `llama-3.3-70b-specdec` model. Key parameters are set to customize the response.

In [47]:
# AUGMENTED GENERATION


# Generate a chat completion
completion = client.chat.completions.create(
    model="llama-3.3-70b-specdec",
    messages=[
        {"role": "user", "content": f"answer the {query} with {response_chunks}" }
        # {"role": "user", "content": f"explain quantum mechanics for a person who is not from science background" }
        # {"role": "user", "content": f"what is tax liability per latest budget 2025" }
        # GENERATION OF RESPONSE
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None
)

print(completion.choices[0].message.content)

**Foundation Language Models vs. Fine-Tuned Language Models: A Beginner's Guide**

When it comes to Large Language Models (LLMs), there are two main types: Foundation Language Models and Fine-Tuned Language Models. In this guide, we'll explore the differences between these two types of models and provide an overview of Fine-Tuned Language Models that specialize in performing tasks in a narrowly scoped domain.

**Foundation Language Models**

Foundation Language Models, such as MT-NLG and GPT-3, are trained on vast amounts of data and can perform a wide variety of Natural Language Processing (NLP) tasks. These models are designed to be general-purpose and can generate text, answer questions, translate languages, and more. They're often referred to as "foundation" models because they provide a foundation for a wide range of NLP applications.

**Fine-Tuned Language Models**

Fine-Tuned Language Models, on the other hand, are specialized models that are trained on a specific dataset to per

## Key Parameters Explained
- **`model`**: Specifies the model to use. In this case, `llama-3.3-70b-specdec`.
- **`messages`**: The input prompt, structured as a list of message objects. Each message has:
  - **`role`**: Indicates whether the message is from the `user` or `assistant`.
  - **`content`**: The actual input text provided by the user.
- **`temperature`**: Controls randomness in the response. A value of 1 generates more diverse responses.
- **`max_tokens`**: Sets the maximum number of tokens (words or subwords) in the output.
- **`top_p`**: Implements nucleus sampling for better output diversity.
- **`stream`**: When `True`, streams the response in real-time.
- **`stop`**: Specifies stopping conditions for the response. Set to `None` for no specific stopping conditions.

In [ ]:
print(completion.choices[0].message.content)

Quantum mechanics! One of the most fascinating and mind-bending branches of physics. I'll try to break it down in a way that's easy to understand, but keep in mind that quantum mechanics is a complex and abstract subject.

**What is Quantum Mechanics?**

Quantum mechanics is a branch of physics that studies the behavior of matter and energy at the smallest scales, such as atoms and subatomic particles like electrons and photons. At these scales, the classical laws of physics, like Newton's laws, don't apply, and strange, seemingly random phenomena start to occur.

**Key Principles:**

1. **Wave-Particle Duality**: Quantum objects, like electrons, can exhibit both wave-like and particle-like behavior depending on how they're observed. This challenges our classical notion of objects being either waves or particles.
2. **Uncertainty Principle**: It's impossible to know certain properties of a quantum object, like its position and momentum, simultaneously with infinite precision. The more 

In [ ]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq


client = Groq(api_key="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",)


chat_completion = client.chat.completions.create(
    messages=[
        { "role": "user","content": "Explain the importance of fast language models",}
    ],
    model="llama-3.3-70b-versatile",
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be understood from several perspectives:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and speech recognition systems. These models can quickly process and respond to user input, providing a seamless and interactive experience.
2. **Low-Latency Requirements**: Many industries, such as finance, healthcare, and customer service, require fast and efficient language processing to meet low-latency requirements. Fast language models can help reduce the time it takes to process and respond to critical information.
3. **Scalability**: Fast language models can handle large volumes of data and scale to meet the demands of big data and high-traffic applications. This is particularly important for applications that need to process and analyze vast amounts of text data.
4. **Embedded Systems**: Fast language mode

## Example Output
Upon running the code, the response is streamed from the model in real-time, explaining **why MLOps is required** in a simple manner suitable for a 10-year-old child.